In [1]:
# CS 340 Project Two: Grazioso Salvare Dashboard
# Student: Milfred

# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import my CRUD Python module from Project One
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Updated with my username and password and CRUD Python module name

username = "aacuser"
password = "Password123!"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# Class read method must support return of list object and accept projection json input
# Sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'], inplace=True)

# Debug - verify data loading
print(f"Loaded {len(df.to_dict(orient='records'))} records")
print(f"Available columns: {list(df.columns)}")

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Add in Grazioso Salvare's logo with error handling
def encode_image_safe(image_filename):
    try:
        encoded_image = base64.b64encode(open(image_filename, 'rb').read())
        return f'data:image/png;base64,{encoded_image.decode()}'
    except FileNotFoundError:
        print(f"Logo file {image_filename} not found. Dashboard will display without logo.")
        return None

# Try to load the Grazioso Salvare logo
image_filename = 'Grazioso Salvare Logo.png'
encoded_image_src = encode_image_safe(image_filename)

app.layout = html.Div([
    # Header section with logo and unique identifier
    html.Div([
        # Grazioso Salvare logo (if available)
        html.Img(
            src=encoded_image_src if encoded_image_src else '',
            style={
                'height': '80px',
                'width': 'auto',
                'margin-right': '20px',
                'display': 'inline-block' if encoded_image_src else 'none'
            }
        ),
        html.Div([
            html.Center(html.B(html.H1('Milfred - CS-340 Grazioso Salvare Dashboard'))),
            html.Center(html.P('Student: Milfred | Project Two | Animal Rescue Dashboard')),
            html.Center(html.P('Austin Animal Center - Search and Rescue Dog Identification'))
        ])
    ], style={'background-color': '#f8f9fa', 'padding': '20px', 'margin-bottom': '20px'}),
    
    html.Hr(),
    
    # Interactive filtering options section
    html.Div([
        html.H3('Rescue Type Filters', style={'color': '#2c5aa0'}),
        html.P('Select a rescue type to filter available dogs:'),
        
        # Radio buttons for different rescue types
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': ' Reset (Show All)', 'value': 'reset'},
                {'label': ' Water Rescue', 'value': 'water'},
                {'label': ' Mountain or Wilderness Rescue', 'value': 'mountain'},
                {'label': ' Disaster Rescue or Individual Tracking', 'value': 'disaster'}
            ],
            value='reset',
            labelStyle={'display': 'inline-block', 'margin-right': '20px', 'margin-bottom': '10px'},
            style={'margin-bottom': '20px'}
        ),
        
        # Filter information display
        html.Div(id='filter-info', style={
            'background-color': '#e8f4fd',
            'padding': '15px',
            'border-radius': '5px',
            'margin-bottom': '20px'
        })
    ], style={'margin': '20px', 'padding': '20px', 'border': '1px solid #ddd', 'border-radius': '10px'}),
    
    html.Hr(),
    
    # Interactive data table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        
        # Set up the features for interactive data table to make it user-friendly
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
        
        # Table styling
        style_table={'overflowX': 'auto'},
        style_cell={
            'textAlign': 'left',
            'padding': '10px',
            'fontFamily': 'Arial'
        },
        style_header={
            'backgroundColor': '#2c5aa0',
            'color': 'white',
            'fontWeight': 'bold'
        },
        style_data_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': '#f8f9fa'
            }
        ]
    ),
    
    html.Br(),
    html.Hr(),
    
    # This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display': 'flex'},
         children=[
        html.Div([
            html.H3('Breed Distribution Chart'),
            html.Div(id='graph-id')
        ],
            className='col s12 m6',
            style={'width': '48%', 'margin-right': '2%'}
        ),
        html.Div([
            html.H3('Animal Location Map'),
            html.Div(id='map-id')
        ],
            className='col s12 m6',
            style={'width': '48%'}
        )
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Callback for updating filter information
@app.callback(
    Output('filter-info', 'children'),
    [Input('filter-type', 'value')]
)
def update_filter_info(filter_value):
    if filter_value == 'reset':
        return html.Div([
            html.H4("All Animals", style={'margin': '0 0 10px 0'}),
            html.P("Showing all animals in the database")
        ])
    elif filter_value == 'water':
        return html.Div([
            html.H4("Water Rescue Dogs", style={'margin': '0 0 10px 0'}),
            html.P("Breeds: Labrador Retriever Mix, Chesapeake Bay Retriever, Newfoundland"),
            html.P("Sex: Intact Female | Age: 26-156 weeks")
        ])
    elif filter_value == 'mountain':
        return html.Div([
            html.H4("Mountain/Wilderness Rescue Dogs", style={'margin': '0 0 10px 0'}),
            html.P("Breeds: German Shepherd, Alaskan Malamute, Old English Sheepdog, Siberian Husky, Rottweiler"),
            html.P("Sex: Intact Male | Age: 26-156 weeks")
        ])
    elif filter_value == 'disaster':
        return html.Div([
            html.H4("Disaster Rescue Dogs", style={'margin': '0 0 10px 0'}),
            html.P("Breeds: Doberman Pinscher, German Shepherd, Golden Retriever, Bloodhound, Rottweiler"),
            html.P("Sex: Intact Male | Age: 20-300 weeks")
        ])

# Main callback to update data table with MongoDB queries
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    # Add code to filter interactive data table with MongoDB queries
    # Use the CRUD module to query the database based on rescue type
    
    if filter_type == 'water':
        # Water rescue query
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'mountain':
        # Mountain/wilderness rescue query
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'disaster':
        # Disaster rescue query
        query = {
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }
    else:  # reset
        # Show all animals
        query = {}
    
    # Use CRUD module to execute the query
    try:
        results = db.read(query)
        df_filtered = pd.DataFrame(results)
        
        if not df_filtered.empty and '_id' in df_filtered.columns:
            df_filtered.drop(columns=['_id'], inplace=True)
        
        return df_filtered.to_dict('records')
    except Exception as e:
        print(f"Query error: {e}")
        return df.to_dict('records')  # Return original data if query fails

# Display the breeds of animal based on quantity represented in the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    # Add code for chart of choice (breed distribution pie chart)
    if viewData is None or len(viewData) == 0:
        return [html.P("No data available for chart")]
    
    # Convert to DataFrame for chart creation
    dff = pd.DataFrame.from_dict(viewData)
    
    # Create breed distribution chart
    if 'breed' in dff.columns:
        breed_counts = dff['breed'].value_counts().head(10)
        
        return [
            dcc.Graph(
                figure=px.pie(
                    values=breed_counts.values,
                    names=breed_counts.index,
                    title=f'Breed Distribution ({len(dff)} animals)'
                )
            )
        ]
    else:
        return [html.P("Breed data not available")]

# This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        selected_columns = []
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback will update the geo-location chart for the selected data entry
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None:
        # Return default Austin map
        return [
            dl.Map(
                style={'width': '100%', 'height': '400px'},
                center=[30.75, -97.48],
                zoom=10,
                children=[
                    dl.TileLayer(id="base-layer-id"),
                    dl.Marker(
                        position=[30.75, -97.48],
                        children=[
                            dl.Tooltip("Austin Animal Center"),
                            dl.Popup([html.H1("Austin Animal Center"), html.P("No animal selected")])
                        ]
                    )
                ]
            )
        ]
    
    dff = pd.DataFrame.from_dict(viewData)
    
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]
    
    # Safety check
    if row >= len(dff):
        row = 0
    
    # Get coordinates with error handling
    try:
        # Try to get lat/long from data
        if 'location_lat' in dff.columns and 'location_long' in dff.columns:
            lat = dff.iloc[row]['location_lat'] if pd.notna(dff.iloc[row]['location_lat']) else 30.75
            lon = dff.iloc[row]['location_long'] if pd.notna(dff.iloc[row]['location_long']) else -97.48
        else:
            # Fallback to column indices as in original template
            lat = dff.iloc[row, 13] if len(dff.columns) > 13 and pd.notna(dff.iloc[row, 13]) else 30.75
            lon = dff.iloc[row, 14] if len(dff.columns) > 14 and pd.notna(dff.iloc[row, 14]) else -97.48
        
        # Get animal info
        breed = dff.iloc[row, 4] if len(dff.columns) > 4 else "Unknown"
        name = dff.iloc[row, 9] if len(dff.columns) > 9 else "Unknown"
        
    except (IndexError, KeyError):
        # Austin TX coordinates as fallback
        lat, lon = 30.75, -97.48
        breed = "Unknown"
        name = "Unknown"
    
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(
            style={'width': '100%', 'height': '400px'},
            center=[lat, lon],
            zoom=15,
            children=[
                dl.TileLayer(id="base-layer-id"),
                # Marker with tool tip and popup
                dl.Marker(
                    position=[lat, lon],
                    children=[
                        dl.Tooltip(str(breed)),
                        dl.Popup([
                            html.H1("Animal Name"),
                            html.P(str(name))
                        ])
                    ]
                )
            ]
        )
    ]

# Run the dashboard
app.run_server(debug=True)

Successfully connected to MongoDB database: AAC
Read operation successful: Retrieved 60003 document(s)
Loaded 60003 records
Available columns: ['animal_id', 'name', 'breed', 'outcome_type', 'rec_num', 'age_upon_outcome', 'animal_type', 'color', 'date_of_birth', 'datetime', 'monthyear', 'outcome_subtype', 'sex_upon_outcome', 'location_lat', 'location_long', 'age_upon_outcome_in_weeks']
Logo file Grazioso Salvare Logo.png not found. Dashboard will display without logo.
Dash app running on http://127.0.0.1:29391/
Read operation successful: Retrieved 60003 document(s)
Read operation successful: Retrieved 60003 document(s)
Read operation successful: Retrieved 102 document(s)
Read operation successful: Retrieved 30 document(s)
Read operation successful: Retrieved 102 document(s)
